In [1]:
import mysql.connector
import psycopg2
import pandas as pd
import time
from config import host, user, password
from config2 import user1, password1

In [2]:
start_time = time.time()

In [3]:
### Establishing MySQL and PostgreSQL Connections

#MySQL Connection
try:
    mysql_db = mysql.connector.connect(host=host,user=user,password=password,db="eBooks2go")
    mycursor = mysql_db.cursor()
    print("MySQL Connection Established")
except mysql.connector.Error as e:
    print("Unable to Connect: ",format(e))

#PostgreSQL Connection
try:
    pg_db = psycopg2.connect(host=host,user=user1,password=password1,database="staging")
    pg_db.autocommit = True
    pgcursor = pg_db.cursor()
    print("PostgreSQL Connection Established")
except psycopg2.OperationalError as e:
    print("Unable to Connect: ",format(e))

#PostgreSQL Connection
try:
    pg_db1 = psycopg2.connect(host=host,user=user1,password=password1,database="staging")
    pg_db1.autocommit = True
    pgcursor1 = pg_db1.cursor()
    print("PostgreSQL Connection Established")
except psycopg2.OperationalError as e:
    print("Unable to Connect: ",format(e))

MySQL Connection Established
PostgreSQL Connection Established
PostgreSQL Connection Established


In [ ]:
new_records_source = 0
new_records_target = 0
count_failed = 0

#Selects data from table
slt_cmd ="Select * from eBooks2go.CRM_Customer"
mycursor.execute(slt_cmd)

#Extracting column names 
col_names_lst = [i[0] for i in mycursor.description] 
col_names = ', '.join(col_names_lst)+', Created_Date'+', Updated_Date'
#     print(col_names)

#Inserting data from MySQL into PostgreSQL
placeholders = ', '.join(['%s'] * (len(col_names_lst)+2) ) 

#Inserting data into PostgreSQL
insrt_cmd = "INSERT INTO stg_CRM_CUSTOMER("+col_names+") " \
            "VALUES ( %s )" % (placeholders)
#     print(insrt_cmd)

for row in mycursor:
    try:
        new_records_source = mycursor.rowcount
        pgcursor.execute(insrt_cmd,row+tuple([time.strftime("%Y/%m/%d %H:%M")])+tuple([time.strftime("%Y/%m/%d %H:%M")]))
        new_records_target += 1
        
    except Exception as e:
        count_failed += 1  

#Log_Audit
mycursor.execute('SELECT count(customer_id) FROM eBooks2go.CRM_Customer')
Total_Records_from_source = mycursor.fetchone()

pgcursor.execute('SELECT count(customer_id) FROM stg_CRM_Customer')
Total_Records_from_target = pgcursor.fetchone()

insrt_log = "INSERT INTO LOG_DIM (Phase, Source_Table_Name, Target_Table_Name, \
Total_Records_Source_Table, Total_Records_Target_Table, New_Records_Source, \
New_Records_Target,Status, Remarks, Execution_time) Values ('LtoS','eBooks2go.CRM_CUSTOMER',\
'stg_CRM_CUSTOMER'," +str(Total_Records_from_source)+","+str(Total_Records_from_target)+str(new_records_source)\
+","+str(new_records_target)+",'Completed','" +str(count_failed)+" Records Failed.Historical Data Update','"\
+str(round(time.time() - start_time,2))+" seconds')"
pgcursor.execute(insrt_log)

mysql_db.close()
pg_db.close()